# Myocardial Infraction Complications Analysis

## 0. Introduction

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## 1. Preprocessing

In [2]:
%store -r preprocessed_data
preprocessed_data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,ZSN_A,nr_11,...,L_BLOOD,TIME_B_S,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
0,0.340030,0.785397,-0.661077,1.595975,0.743926,1.015624,0.564308,-0.193621,-0.278129,-0.171499,...,1.270216,0.537529,2.653402,-0.637667,-0.373961,0.700877,0.618154,-1.684847,-0.137243,2.005267
1,-0.566563,0.785397,0.560478,-0.888818,-1.203867,-1.491829,0.564308,-0.193621,-0.278129,-0.171499,...,-1.371753,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,-0.498687
2,0.158711,0.785397,-0.661077,-0.888818,0.743926,-0.238103,0.564308,-0.193621,-0.278129,-0.171499,...,1.036118,-0.178374,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,2.005267
3,0.702667,0.785397,-0.661077,1.595975,0.743926,-0.238103,-1.278015,-0.193621,-0.278129,-0.171499,...,-0.335031,1.611383,-0.376875,-0.637667,-0.373961,0.700877,0.618154,-1.684847,-0.137243,2.005267
4,0.158711,-1.273242,-0.661077,-0.888818,-1.203867,1.015624,0.564308,-0.193621,1.261282,-0.171499,...,-0.535687,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,-1.684847,-0.137243,-0.498687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,1.065304,-1.273242,-0.661077,-0.888818,0.743926,-0.238103,-1.278015,-0.193621,-0.278129,-0.171499,...,2.340381,-0.536326,-0.376875,-0.637667,-0.373961,-1.426785,-1.617721,-1.684847,-0.137243,2.005267
1186,0.068052,0.785397,-0.661077,-0.888818,-1.203867,-1.491829,-1.278015,-0.193621,-0.278129,-0.171499,...,-0.335031,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,-0.498687
1187,0.068052,0.785397,-0.661077,1.595975,0.743926,-0.238103,-1.278015,-0.193621,1.261282,-0.171499,...,0.868904,-0.894277,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,2.005267
1188,0.430689,0.785397,0.560478,1.595975,0.743926,-0.238103,-1.278015,-0.193621,1.261282,-0.171499,...,0.501035,-0.894277,-0.376875,-0.637667,-0.373961,0.700877,-1.617721,0.593526,-0.137243,-0.498687


## 2. Feature Engineering